In [1]:
import os, sys
import json
import re
from tqdm import tqdm

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import dump_svmlight_file
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from skmultilearn.model_selection import iterative_train_test_split
import json
import sklearn
import pickle
import os
import sys
from spiral import ronin

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from clean_text import cleaned_text

In [417]:
with open('/home/chentianyu/libminer/advisory/combined_reports.json', 'r') as f:
    vul_lib = json.load(f)
with open('/home/chentianyu/libminer/advisory/new_cve_id_v3.json', 'r') as f:
    used = set(json.load(f))
vul_lib = [vuln for vuln in vul_lib if vuln['cve_id'] in used]
for vuln in vul_lib:
    if len(vuln['labels'][0].split(':')) == 2:
        vuln['labels'] = ['maven:'+lib for lib in vuln['labels']]
        
target = [(vuln['cve_id'], vuln) for vuln in vul_lib]
target.sort()
vul_lib = [item[1] for item in target]
length = len(vul_lib)
# train = vul_lib[:length * 2 // 5] + vul_lib[length * 3 // 5:length * 4 // 5]
# valid = vul_lib[length * 2 // 5:length * 3 // 5]
# test = vul_lib[length * 4 // 5:]
train = vul_lib[length * 1 // 5:length * 4 // 5]
valid = vul_lib[:length * 1 // 5]
test = vul_lib[length * 4 // 5:]

with open('/home/chentianyu/libminer/advisory/zero_names.json', 'r') as f:
    zero_names = set(json.load(f))
with open('/home/chentianyu/libminer/advisory/full_names.json', 'r') as f:
    full_names = set(json.load(f))

In [ ]:
test[333]

In [418]:
test = [vuln for vuln in test if vuln['cve_id'] in zero_names]
len(test)

452

In [312]:
with open('/home/chentianyu/libminer/maven_corpus_new.json', 'r') as f:
    maven_corpus = json.load(f)

In [313]:
vul_lib_labels = list(set([label[6:] for vuln in vul_lib for label in vuln['labels']]))

In [314]:
maven_labels = [lib['name'] for lib in maven_corpus]

In [339]:
use_maven = True

In [419]:
vul_lib_dir = '/home/chentianyu/Chronos/zestxml/GZXML-Datasets/vul_maven_v3_zero/' if use_maven else \
'/home/chentianyu/Chronos/zestxml/GZXML-Datasets/vul_lib_v3_3/'

In [420]:
if not os.path.isdir(vul_lib_dir):
    os.mkdir(vul_lib_dir)
vul_lib_dir

'/home/chentianyu/Chronos/zestxml/GZXML-Datasets/vul_maven_v3_zero/'

The following is Xf.txt

In [421]:
vuln_corpus = [' '.join(cleaned_text(vuln['desc'])) for vuln in vul_lib]

In [422]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorizer.fit(vuln_corpus)

TfidfVectorizer(ngram_range=(1, 2))

In [423]:
tfidf_vectorizer = vectorizer

In [424]:
with open(os.path.join(vul_lib_dir, 'Xf.txt'), 'w', encoding='utf-8') as wr:
    for key in sorted(vectorizer.vocabulary_, key=vectorizer.vocabulary_.get):
        wr.write(key + "\n")

In [425]:
def get_row_Xf():
    num_lines = sum(1 for line in open(os.path.join(vul_lib_dir, 'Xf.txt')))
    return str(num_lines)

In [426]:
XF_ROW = get_row_Xf()
XF_ROW

'76675'

The following is Yf.txt

In [427]:
label_columns = [item for item in maven_labels] if use_maven else \
    [item for item in vul_lib_labels]
label_to_index = {label_columns[i]:i for i in range(len(label_columns))}

for i in range(len(label_columns)):
    splitted_label = re.split('-|\.|:', label_columns[i])
    for word in splitted_label:
        label_columns[i] = label_columns[i] + " " + word

In [428]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
vectorizer.fit(label_columns)

TfidfVectorizer()

In [429]:
label_vectorizer = vectorizer

In [430]:
with open(os.path.join(vul_lib_dir, 'Yf.txt'), "w", encoding="utf-8") as wr:
    for key in sorted(vectorizer.vocabulary_, key=vectorizer.vocabulary_.get):
        wr.write(key + "\n")

In [431]:
def get_row_Yf(): 
    num_lines = sum(1 for line in open(os.path.join(vul_lib_dir, 'Yf.txt')))
    return str(num_lines)

In [432]:
YF_ROW = get_row_Yf()
YF_ROW

'95189'

The following is Y_Yf.txt

In [433]:
with open(os.path.join(vul_lib_dir, 'Y_Yf.txt'), "w", encoding="utf-8") as wr:
    # header is number of labels SPACE number of features (i.e., numrows of Yf.txt)
    wr.write(f'{len(label_columns)} {YF_ROW}\n')
    # wr.write("2817 "+YF_ROW+"\n")
    for label in tqdm(label_columns):
        sparse_mat = vectorizer.transform([label])
        value = sparse_mat.data
        indices = sparse_mat.indices
        sorted_value = [x for _, x in sorted(zip(indices, value))]
        sorted_indices = sorted(indices)
        # printing the tfidf values
        to_print = ""
        for i in range(0, len(sorted_value)):
            to_print = to_print + sorted_indices[i].__str__() + ":" + sorted_value[i].__str__() + " "
        to_print = to_print[:-1] + "\n"
        wr.write(to_print)

100%|██████████| 311233/311233 [03:38<00:00, 1423.63it/s]


The following is trn_Y_Yf.txt

In [434]:
list_labels = list(set([label[6:] for vuln in train for label in vuln['labels']]))

In [435]:
with open(os.path.join(vul_lib_dir, 'trn_Y_Yf.txt'), "w", encoding="utf-8") as wr:
    # header is number of labels SPACE number of features (i.e., numrows of Yf.txt)
    wr.write(len(list_labels).__str__() + " "+YF_ROW+"\n")
    for label in list_labels:
        sparse_mat = vectorizer.transform([label])
        value = sparse_mat.data
        indices = sparse_mat.indices
        sorted_value = [x for _, x in sorted(zip(indices, value))]
        sorted_indices = sorted(indices)
        # printing the tfidf values
        to_print = ""
        for i in range(0, len(sorted_value)):
            to_print = to_print + sorted_indices[i].__str__() + ":" + sorted_value[i].__str__() + " "
        to_print = to_print[:-1] + "\n"
        wr.write(to_print)

The following is trn_X_Xf.txt and tst_X_Xf.txt

In [436]:
train_desc = [' '.join(cleaned_text(vuln['desc'])) for \
              vuln in train if len(vuln['labels']) > 0]
valid_desc = [' '.join(cleaned_text(vuln['desc'])) for \
              vuln in valid if len(vuln['labels']) > 0]
test_desc = [' '.join(cleaned_text(vuln['desc'])) for \
              vuln in test if len(vuln['labels']) > 0]

In [437]:
train_X = tfidf_vectorizer.transform(train_desc)
valid_X = tfidf_vectorizer.transform(valid_desc)
test_X = tfidf_vectorizer.transform(test_desc)

In [438]:
xf_wr = open(os.path.join(vul_lib_dir, 'trn_X_Xf.txt'), "w", encoding="utf-8")
xy_wr = open(os.path.join(vul_lib_dir, 'trn_X_Y.txt'), "w", encoding="utf-8")
xf_wr.write(f'{len(train_desc)} {XF_ROW}\n')
xy_wr.write(f'{len(train_desc)} {len(label_columns)}\n')
for index in range(train_X.shape[0]):
    xf_target =[f'{idx}:{p}' for idx, p in zip(train_X[index].indices, train_X[index].data)]
    xf_wr.write(' '.join(xf_target)+'\n')

    label_target = [f'{label_to_index[label[6:]]}:{1.00000}' for label in train[index]['labels']]
    xy_wr.write(' '.join(label_target) + '\n')
xf_wr.close()
xy_wr.close()

In [439]:
xf_wr = open(os.path.join(vul_lib_dir, 'tst_X_Xf.txt'), "w", encoding="utf-8")
xy_wr = open(os.path.join(vul_lib_dir, 'tst_X_Y.txt'), "w", encoding="utf-8")
xf_wr.write(f'{len(test_desc) * 3} {XF_ROW}\n')
xy_wr.write(f'{len(test_desc) * 3} {len(label_columns)}\n')
for repeat in range(3):
    for index in range(test_X.shape[0]):
        xf_target =[f'{idx}:{p}' for idx, p in zip(test_X[index].indices, test_X[index].data)]
        xf_wr.write(' '.join(xf_target)+'\n')

        label_target = [f'{label_to_index[label[6:]]}:{1.00000}' for label in test[index]['labels']]
        xy_wr.write(' '.join(label_target) + '\n')
xf_wr.close()
xy_wr.close()

In [440]:
XF_ROW

'76675'

In [58]:
train_X[0].indices

array([45453, 45445, 44219, 43976, 43563, 41440, 41419, 35833, 35805,
       35660, 35567, 34300, 34278, 33830, 33819, 33209, 33198, 32518,
       32515, 31615, 31610, 31374, 31370, 26856, 26849, 25401, 25370,
       22150, 22149, 21934, 21933, 18058, 18044, 13451, 13450,  6933,
        6888,  6886,  5191,  5142,  3585,  3573,  3308,  3300,  1582,
        1506], dtype=int32)

In [59]:
train_X[0].data

array([0.18496186, 0.11649603, 0.09898618, 0.04823711, 0.07689307,
       0.101143  , 0.07614777, 0.18496186, 0.10410068, 0.16960882,
       0.07938643, 0.18496186, 0.11322593, 0.18496186, 0.12425293,
       0.05722962, 0.04837613, 0.18496186, 0.15721347, 0.18496186,
       0.13396017, 0.18496186, 0.15164692, 0.18496186, 0.13755992,
       0.18496186, 0.12156482, 0.18496186, 0.18496186, 0.18496186,
       0.18496186, 0.18496186, 0.11700525, 0.18496186, 0.16466624,
       0.18496186, 0.18496186, 0.23102288, 0.14186043, 0.08432901,
       0.18496186, 0.1350963 , 0.10270716, 0.04379441, 0.05899677,
       0.04261674])

In [34]:
train_X.shape

(1668, 46547)

'org.jenkins-ci.plugins:htmlpublisher org jenkins ci plugins htmlpublisher'

In [14]:
len(label_columns)

2095

In [ ]:
list_labels[0]